In [1]:
from pystac_client import Client
from odc.stac import load

In [2]:
latlon_min = (-15.713818618098927, 166.64917826697013)
latlon_max = (-15.099763342631364, 167.32464476929673)

bbox = [latlon_min[1], latlon_min[0], latlon_max[1], latlon_max[0]]

In [4]:
catalog = "https://stac.staging.digitalearthpacific.io/"
client = Client.open(catalog)

In [ ]:
items = client.search(collections="dep_s2_geomad", bbox=bbox).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(items, bbox=bbox, resolution=10, chunks={"x": 2048, "y": 2048})
data

In [11]:
import rasterio

with rasterio.Env(AWS_NO_SIGN_REQUEST="YES"):
    loaded = data[["red", "green", "blue"]].compute()

In [ ]:
loaded.odc.geobox.extent.centroid.coords[0]

In [ ]:
import folium

m = folium.Map(location=loaded.odc.geobox.extent.centroid.to_crs("epsg:4326").coords[0][::-1], zoom_start=10)

loaded.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m)

m

In [27]:
m.save("index.html")